# Gesture recognition
# Kaszti Dávid
# Kézi gesztusfelismerés PowerPoint vezérélésére
# Hand gesture recognition for controlling PowerPoint
# CNN tanítás Python Jupyter notebook

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers.optimizer_v2.adam import Adam
from keras_preprocessing.image import ImageDataGenerator
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet_v2 import MobileNetV2
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=10,
                                 zoom_range=0.1)
img_size = 64
train_dir = r'C:\Users\admin\Desktop\modellek\v4\6464_gray\training\train'
test_dir = r'C:\Users\admin\Desktop\modellek\v4\6464_gray\training\test'

In [ ]:
def set_database(traindir, testdir):
    train_datagen = datagen.flow_from_directory(
            traindir,
            target_size=(img_size, img_size),
            batch_size=64,
            class_mode='categorical',
            color_mode='grayscale',
          )

    val_datagen = datagen.flow_from_directory(
            testdir,
            target_size=(img_size, img_size),
            batch_size=64,
            color_mode='grayscale',
            class_mode='categorical',
        )
    return train_datagen, val_datagen

In [ ]:
train_datagen, val_datagen = set_database(train_dir, test_dir);

In [ ]:

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu',
                         input_shape=(img_size, img_size, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(6, activation="softmax"))
model.compile(optimizer="adam", loss='categorical_crossentropy',
                      metrics=['accuracy', metrics.Recall(), metrics.Precision()])

model.summary()
keras.utils.plot_model(model, "cnn_model3.png", show_shapes=True, dpi=128)

In [ ]:
batch_size = 64
epochs = 20
es = EarlyStopping(monitor='val_loss', min_delta=0, patience = 5)
log = model.fit(train_datagen, epochs=epochs, batch_size=batch_size, validation_data=val_datagen, callbacks=[es])

In [ ]:
plt.plot(log.history['accuracy'])
plt.plot(log.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(log.history['loss'])
plt.plot(log.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
val_datagen = datagen.flow_from_directory(
            test_dir,
            target_size=(img_size, img_size),
            batch_size=64,
            color_mode='grayscale',
            class_mode='categorical',
            shuffle = False
)

In [ ]:
Y_pred = model.predict(val_datagen)
y_pred = np.argmax(Y_pred, axis=1)
print('Metrics')
target_names = ['A', 'B', 'H', 'L', 'P', 'Q']
print(classification_report(val_datagen.classes, y_pred, target_names=target_names))

In [ ]:
conf_mat = confusion_matrix(val_datagen.classes, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=target_names)
disp.plot()
plt.show()

In [ ]:
model.save("test_15.h5")